In [ ]:
from gensim.models import FastText as fText
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('./wiki.en.vec')
model_words = list(model.wv.vocab)     #Loading the model words
model_bin = fText.load_fasttext_format("./wiki.en.bin")

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  


We loaded the model using gensim. 'model_words' contains the word in which are there in the dict of pre-trained model.

In [4]:
from nltk import SnowballStemmer
from nltk.corpus import stopwords

stop_words = stopwords.words('english')      #loading the stopwords
stemmer = SnowballStemmer("english")         #loading the stemmer

delimiter = [',', ':','!', '@','&','$','.','/',']','[']     #defining a set of delimeters

In [3]:
import csv

data_encoding = dict()
flag = 0

with open("./ukb_coding19.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
    	if (flag==0):
    		flag = 1
    		continue
    	a = row[1].split(' ',1)[1]
    	data_encoding[row[0]]=a

In [4]:
app_syn = dict()
cli_info = dict()

with open("./icd_info4.csv") as fd:
    rd = csv.reader(fd, delimiter=",", quotechar='"')
    for row in rd:
    	row[0] = row[0].split('-')[0]
    	try:
    		row[0] = row[0].split('.')[0] + row[0].split('.')[1]
    	except:
    		row[0] = row[0].split('.')[0]
    	cli_info[row[0]] = row[3]
    	app_syn[row[0]] = row[4]

We loaded the approximate synonyms, clincal information and ICD code description in three different dict file.

In [1]:
from nltk import word_tokenize, sent_tokenize
import numpy as np

def process(value):       #This method gives a single vector corresponding to the value passed.
	word_count = 0
	value_vec = np.array([0] * 300)
	for word in word_tokenize(value):
		word = word.lower()
		if(word in model_words and word not in stop_words and word not in delimiter):
			value_vec = value_vec + model.wv[word]
			word_count = word_count + 1
		elif(stemmer.stem(word) in model_words and stemmer.stem(word) not in stop_words and stemmer.stem(word) not in delimiter):
			value_vec = value_vec + model.wv[stemmer.stem(word)]
			word_count = word_count + 1
		elif(word not in stop_words and word not in delimiter):
			value_vec = value_vec + model_bin.wv[word]
			word_count = word_count + 1

	if(word_count>0):
		value_vec = value_vec/word_count

	return value_vec

In [5]:
icd_code_vec = dict()     #for storing vector corresponding to each ICD code
for key, value in data_encoding.items():
    icd_code_desc = value
    try:
        if(len(app_syn[key])>0):
            icd_code_desc = icd_code_desc + " " + app_syn[key]
        if(len(cli_info[key])>0):
            icd_code_desc = icd_code_desc + " " + cli_info[key]
    except:
        icd_code_desc = icd_code_desc
    icd_code_vec[key] = process(icd_code_desc)

NameError: name 'model_words' is not defined